In [1]:
from pycaret.regression import *
import pandas as pd

# 1. 데이터 불러오기
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [2]:
# 2. HLM을 위한 환경 설정
setup_data_HLM = setup(data = train_data, 
                       target = 'HLM', 
                       text_features = ['SMILES'], 
                       ignore_features = ['id', 'MLM'],
                       session_id = 123,
                       # use_gpu = True,
                       verbose = True
                      )

,Description,Value
0,Session id,123
1,Target,HLM
2,Target type,Regression
3,Original data shape,"(3498, 11)"
4,Transformed data shape,"(3498, 2614)"
5,Transformed train set shape,"(2448, 2614)"
6,Transformed test set shape,"(1050, 2614)"
7,Ignore features,2
8,Numeric features,7
9,Text features,1


In [3]:
# 3. 여러 모델 학습 및 비교
best_HLM = compare_models(n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,27.6971,1051.7870,32.4186,0.1925,1.3503,34.7165,0.5100
catboost,CatBoost Regressor,27.4335,1072.1760,32.7297,0.1771,1.3366,32.7130,1.7570
br,Bayesian Ridge,28.2158,1084.1062,32.9171,0.1675,1.3634,35.1345,1.5770
rf,Random Forest Regressor,27.4267,1104.4679,33.2045,0.1520,1.3303,33.4587,1.1360
lasso,Lasso Regression,28.7823,1109.4979,33.3009,0.1481,1.3801,36.1930,0.1260
llar,Lasso Least Angle Regression,28.7823,1109.4950,33.3009,0.1481,1.3801,36.1929,0.0860
en,Elastic Net,29.0196,1116.8913,33.4125,0.1424,1.3867,36.6950,0.0880
ada,AdaBoost Regressor,29.5058,1128.6978,33.5881,0.1333,1.3944,36.1446,0.3810
xgboost,Extreme Gradient Boosting,27.9711,1147.8577,33.8552,0.1192,1.3441,33.6854,1.8750
huber,Huber Regressor,28.6330,1149.7985,33.8926,0.1177,1.3615,34.7048,0.4240


Processing:   0%|          | 0/87 [00:00<?, ?it/s]

In [5]:
# 4. 모델 블렌딩
blended_HLM = blend_models(estimator_list = best_HLM)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,27.4182,1047.7316,32.3687,0.1960,1.3977,51.9851
1,27.2712,973.9036,31.2074,0.2349,1.1289,8.4866
2,27.5442,1041.6048,32.2739,0.1909,1.3630,19.2219
3,27.3295,974.1350,31.2111,0.2758,1.3565,21.4589
4,29.3566,1160.3376,34.0637,0.1784,1.5531,73.9557
5,27.4452,1054.6697,32.4757,0.1692,1.2736,10.3850
6,26.5884,974.0363,31.2096,0.2106,1.2322,50.5562
7,26.9525,1053.3628,32.4556,0.1895,1.4106,36.9249
8,27.1564,1025.1470,32.0179,0.2020,1.3070,48.8632


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
# 5. test 데이터를 사용하여 HLM 값 예측
predictions_HLM = predict_model(blended_HLM, data = test_data)

In [7]:
# 6. 예측된 HLM 값을 feature로 추가
train_data['HLM_predicted'] = predict_model(blended_HLM, data=train_data)['prediction_label']
test_data['HLM'] = predictions_HLM['prediction_label']

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,24.5582,818.4376,28.6083,0.3711,1.2813,31.5704


In [8]:
# 7. MLM을 위한 환경 설정
setup_data_MLM = setup(data = train_data, 
                       target = 'MLM', 
                       text_features = ['SMILES'], 
                       ignore_features = ['id'],
                       session_id = 123,
                       # use_gpu = True,
                       verbose = True
                      )

,Description,Value
0,Session id,123
1,Target,MLM
2,Target type,Regression
3,Original data shape,"(3498, 12)"
4,Transformed data shape,"(3498, 2616)"
5,Transformed train set shape,"(2448, 2616)"
6,Transformed test set shape,"(1050, 2616)"
7,Ignore features,1
8,Numeric features,9
9,Text features,1


In [9]:
# 8. 여러 모델 학습 및 비교
best_MLM = compare_models(n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,18.3679,598.6010,24.4335,0.5323,1.2508,27.3297,0.4740
lasso,Lasso Regression,18.7360,604.6817,24.5667,0.5280,1.2547,27.5536,0.0950
en,Elastic Net,18.7433,604.7248,24.5676,0.5280,1.2552,27.5863,0.0880
llar,Lasso Least Angle Regression,18.7360,604.6819,24.5667,0.5280,1.2547,27.5537,0.0840
br,Bayesian Ridge,18.7448,604.6175,24.5656,0.5280,1.2549,27.6226,1.3010
catboost,CatBoost Regressor,18.5095,621.5993,24.9024,0.5144,1.2586,28.7877,1.7690
huber,Huber Regressor,18.5604,625.1972,24.9703,0.5116,1.2576,27.5383,0.4480
rf,Random Forest Regressor,17.8344,625.5094,24.9633,0.5111,1.1836,22.4114,1.0470
ada,AdaBoost Regressor,20.7728,648.9870,25.4589,0.4934,1.3985,37.0040,0.2560
et,Extra Trees Regressor,17.7282,666.5096,25.7554,0.4789,1.1680,18.8219,1.8620


Processing:   0%|          | 0/87 [00:00<?, ?it/s]

In [11]:
# 9. 모델 블렌딩
blended_MLM = blend_models(estimator_list = best_MLM)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,19.4384,650.6751,25.5083,0.4973,1.2917,19.6147
1,18.8336,619.8837,24.8975,0.5103,1.1019,11.5465
2,19.1850,696.6439,26.3940,0.4220,1.3254,31.1227
3,17.4754,541.6684,23.2738,0.5873,1.2166,24.7202
4,17.1362,537.0638,23.1746,0.5806,1.3062,58.4506
5,18.5173,562.3937,23.7148,0.5768,1.2559,20.9263
6,17.8705,549.4957,23.4413,0.5714,1.2159,17.6600
7,18.6146,625.0343,25.0007,0.4962,1.1724,33.7963
8,17.3626,512.7174,22.6433,0.6253,1.0895,16.4573


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [13]:
test_data['HLM'] = predictions_HLM['prediction_label']

In [14]:
# 10. test 데이터를 사용하여 MLM 값 예측
predictions_MLM = predict_model(blended_MLM, data = test_data)

In [15]:
# 11. 최종 예측값을 CSV로 저장
submission = test_data[['id']]
submission['MLM'] = predictions_MLM['prediction_label']
submission['HLM'] = predictions_HLM['prediction_label']
submission.to_csv('submission_pycaret_HLM_MLM.csv', index=False)